In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.linear_model import BayesianRidge, ARDRegression
from sklearn.svm import SVR
from ForecastSimulator2 import Forecastsimulator, forecast_split_classifier
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn.ensemble import BaggingRegressor

from sklearn.preprocessing import StandardScaler
from pandas_datareader import data as web

plt.rcParams['figure.figsize'] = (18, 10)
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.grid'] = True

In [2]:
today = date.today()
start_date = today - relativedelta(years=5)

print(start_date)
data = pd.DataFrame(web.DataReader('MERC', 'stooq', start_date))
data.info()

2018-03-19
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1259 entries, 2023-03-17 to 2018-03-19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1259 non-null   float64
 1   High    1259 non-null   float64
 2   Low     1259 non-null   float64
 3   Close   1259 non-null   float64
 4   Volume  1259 non-null   float64
dtypes: float64(5)
memory usage: 59.0 KB


In [3]:
close = data[['Close']]
close = close.sort_index(ascending = True)
close

,Close
Date,
2018-03-19,11.0048
2018-03-20,11.1737
2018-03-21,11.3456
2018-03-22,10.8793
2018-03-23,10.5769
...,...
2023-03-13,10.1300
2023-03-14,10.1500
2023-03-15,9.8400


In [4]:
dtp=1
ws = 20

X_train, X_test, y_train, y_test, y_val = forecast_split_classifier(close, window_size=ws, dtp=dtp, test_days=250)
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

fs2 = Forecastsimulator(XGBClassifier())
pred = fs2.retrain_classifier(XGBClassifier(), X_train_scaled, X_test_scaled, y_train, y_test, retrain_period=20)
prediction = pred

fs2.simulate_classifier(prediction, y_test, y_val)

0


ValueError: cannot reshape array of size 1 into shape (250)